###### We use neural TTR.

In [1]:
from pprint import pprint
from nu import Type, BType, PType, DepType, Pred, MeetType_n, FunType, InhibitType_n, StringType_n, Ty, iota, gensym_n, nu, and_n, MeetType, or_n, JoinType, labels, Rec, RecType
from neurons import Network, Neuron, Synapse, ActivityPattern, InhibitorySynapse
from utils import show, example, gensym
import random

We define a sheep as a record with a network (its brain), a thirst indicator and a hunger indicator (real numbers, low is hungry or thirsty) and a location in a given terrain.

In [2]:
Real = BType('Real')
Real.learn_witness_condition(lambda n: isinstance(n,float))
Net = BType('Net')
Net.learn_witness_condition(lambda n: isinstance(n,Network))
terrain = []
for i in range(5):
    terrain.append(gensym('loc'))
print(terrain)
Loc = BType('Loc')
Loc.learn_witness_condition(lambda l: l in terrain)
Sheep = RecType({'brain':Net,'food':Real,'water':Real,'loc':Loc})
maud = Rec({'brain':Network(),'food':1.0,'water':1.0,'loc':terrain[0]})
print(Sheep.query(maud))

['loc0', 'loc1', 'loc2', 'loc3', 'loc4']
True


In [3]:
Blue = BType('Blue')
Green = BType('Green')
Brown = BType('Brown')
Swamp = MeetType(Blue,Green)
print(show(Swamp))

(Blue&Green)


In [4]:
Green.judge(terrain[0])
Brown.judge(terrain[1])
Swamp.judge(terrain[2])
Blue.judge(terrain[3])
Green.judge(terrain[4])
print(Green.query(terrain[2]))

True


In [5]:
def DoMoveRight(x):
    current_loc_index = terrain.index(x.loc)
    if current_loc_index < len(terrain)-1:
        x.loc = terrain[current_loc_index+1]
    else:
        x.loc = terrain[0]
    x.food = x.food-0.01
    x.water = x.water-0.01
def DoMoveLeft(x):
    current_loc_index = terrain.index(x.loc)
    if current_loc_index > 0:
        x.loc = terrain[current_loc_index-1]
    else:
        x.loc = terrain[len(terrain)-1]
    x.food = x.food-0.1
    x.water = x.water-0.1
def Alive(x):
    return x.food>0 and x.water>0

DoMoveRight(maud)
print(terrain.index(maud.loc))
print(Alive(maud))

1
True


In [6]:
DoMoveLeft(maud)
terrain.index(maud.loc)

0

In [7]:
def DoEat(x):
    if Swamp.query(x.loc):
        x.food = x.food-0.2
        x.water = x.water-0.2
    elif Green.query(x.loc):
        x.food = x.food+0.1
def DoDrink(x):
    if Swamp.query(x.loc):
        x.food = x.food-0.2
        x.water = x.water-0.2
    elif Blue.query(x.loc):
        x.water = x.water+0.1

In [8]:
Green_n = nu(Green)
Blue_n = nu(Blue)
Brown_n = nu(Brown)
Green_gran = Green_n.add_grandmother(maud.brain)
Blue_gran = Blue_n.add_grandmother(maud.brain)
Brown_gran = Brown_n.add_grandmother(maud.brain)

def DoObserveEnv(x):
    x.brain.inhibit()
    if Green.query(x.loc):
        Green_n.create_n(x.brain)
    if Blue.query(x.loc):
        Blue_n.create_n(x.brain)
    if Brown.query(x.loc):
        Brown_n.create_n(x.brain)

In [9]:
maud.brain.ntrace()
DoObserveEnv(maud)
maud.brain.display_history()

-------  -  -
Green_n  0  1
Blue_n   0  0
Brown_n  0  0
-------  -  -


In [10]:
DoMoveRight(maud)
DoObserveEnv(maud)
maud.brain.display_history()

-------  -  -  -  -
Green_n  0  1  0  0
Blue_n   0  0  0  0
Brown_n  0  0  0  1
-------  -  -  -  -


In [11]:
DoMoveLeft(maud)
DoObserveEnv(maud)
maud.brain.display_history()

-------  -  -  -  -  -  -
Green_n  0  1  0  0  0  1
Blue_n   0  0  0  0  0  0
Brown_n  0  0  0  1  0  0
-------  -  -  -  -  -  -


## Linking neural activity to action

In [12]:
NeuralType2Action = {}

Eat = BType('Eat')
Drink = BType('Drink')
MoveLeft = BType('MoveLeft')
MoveRight = BType('MoveRight')

Eat_n = nu(Eat)
Drink_n = nu(Drink)
MoveLeft_n = nu(MoveLeft)
MoveRight_n = nu(MoveRight)

Eat_gran = Eat_n.add_switch_grandmother(maud.brain)

Drink_gran = Drink_n.add_switch_grandmother(maud.brain)
MoveLeft_gran = MoveLeft_n.add_switch_grandmother(maud.brain)
MoveRight_gran = MoveRight_n.add_switch_grandmother(maud.brain)

NeuralType2Action[Eat_n]=DoEat
NeuralType2Action[Drink_n]=DoDrink
NeuralType2Action[MoveLeft_n]=DoMoveLeft
NeuralType2Action[MoveRight_n]=DoMoveRight

def DoAction(x):
    for k in NeuralType2Action.keys():
        if k.query_n(x.brain):
            NeuralType2Action[k](x)
            InhibitType_n(k).create_n(x.brain)
        
Eat_n.create_n(maud.brain)
print(maud.food)
DoAction(maud)
print(maud.food)
maud.brain.display_history()


0.78
0.88
-----------  -  -  -  -  -  -  -  -
Green_n      0  1  0  0  0  1  1  1
Blue_n       0  0  0  0  0  0  0  0
Brown_n      0  0  0  1  0  0  0  0
Eat_n        2  2  2  2  2  2  1  0
Drink_n      2  2  2  2  2  2  0  0
MoveLeft_n   2  2  2  2  2  2  0  0
MoveRight_n  2  2  2  2  2  2  0  0
-----------  -  -  -  -  -  -  -  -


## Associating pleasure and pain with eating and drinking

In [13]:
Pleasure = BType('Pleasure')
Pain = BType('Pain')
Pleasure_n = nu(Pleasure)
Pain_n = nu(Pain)
Pleasure_gran  = Pleasure_n.add_grandmother(maud.brain)
Pain_gran = Pain_n.add_grandmother(maud.brain)

def DoEat(x):
    if Swamp.query(x.loc):
        x.food = x.food-0.5
        x.water = x.water-0.5
        InhibitType_n(Pleasure_n).create_n(x.brain)
        Pain_n.create_n(x.brain)
    elif Green.query(x.loc):
        x.food = x.food+0.1
        InhibitType_n(Pain_n).create_n(x.brain)
        Pleasure_n.create_n(x.brain)
def DoDrink(x):
    if Swamp.query(x.loc):
        x.food = x.food-0.5
        x.water = x.water-0.5
        InhibitType_n(Pleasure_n).create_n(x.brain)
        Pain_n.create_n(x.brain)
    elif Blue.query(x.loc):
        x.water = x.water+0.1
        InhibitType_n(Pain_n).create_n(x.brain)
        Pleasure_n.create_n(x.brain)
        
NeuralType2Action[Eat_n]=DoEat
NeuralType2Action[Drink_n]=DoDrink

Eat_n.create_n(maud.brain)
print(maud.food)
DoAction(maud)
print(maud.food)
maud.brain.display_history()


0.88
0.98
-----------  -  -  -  -  -  -  -  -  -  -  -
Green_n      0  1  0  0  0  1  1  1  1  1  1
Blue_n       0  0  0  0  0  0  0  0  0  0  0
Brown_n      0  0  0  1  0  0  0  0  0  0  0
Eat_n        2  2  2  2  2  2  1  0  1  1  0
Drink_n      2  2  2  2  2  2  0  0  0  0  0
MoveLeft_n   2  2  2  2  2  2  0  0  0  0  0
MoveRight_n  2  2  2  2  2  2  0  0  0  0  0
Pleasure_n   2  2  2  2  2  2  2  2  0  1  1
Pain_n       2  2  2  2  2  2  2  2  0  0  0
-----------  -  -  -  -  -  -  -  -  -  -  -


## Learning from observation pleasure/pain

In [14]:
def ExistSynapse(from_neuron,to_neuron,dend_num=0):
    if [i for i in from_neuron.synapses if i.to_neuron.neuron is to_neuron 
           and i.to_neuron.dend_num is dend_num]:
        return True
    else:
        return False
    
def HasSynapseTo(from_neuron):
    return set([s.to_neuron.neuron for s in from_neuron.synapses])

def SharedConnections(neurons):
    return list(set.intersection(*[HasSynapseTo(n) for n in neurons]))
    

def Learn0(network):
    pass

def Learn1(network):
    ActivePerception = [n for n in [Green_gran,Blue_gran,Brown_gran] if n.state.axon == 1]
    ActiveAction = [n for n in [Eat_gran,Drink_gran,MoveLeft_gran,MoveRight_gran] 
                    if n.state.axon == 1]
    if len(ActivePerception) is 1:
        if Pleasure_n.query_last_n(network):
            for n in ActiveAction:
                if not ExistSynapse(ActivePerception[0],n):
                    Synapse(ActivePerception[0],n)
                    #print('Pleasure query: '+str(Pleasure_n.query_n(network))+'\n')
                    print(show(ActivePerception[0].name)+' to '+show(n.name)+
                          ' Pleasure: '+show(Pleasure_n.query_last_n(network)))
        elif Pain_n.query_last_n(network):
            for n in ActiveAction:
                if not ExistSynapse(ActivePerception[0],n,1):
                    Synapse(ActivePerception[0],n,1)
    if network.ntracing():
        network.ntrace()
   
 
def Learn2(network):
    ActivePerception = [n for n in [Green_gran,Blue_gran,Brown_gran] if n.state.axon == 1]
    ActiveAction = [n for n in [Eat_gran,Drink_gran,MoveLeft_gran,MoveRight_gran] 
                    if n.state.axon == 1]
    if len(ActivePerception) is 1:
        if Pleasure_n.query_last_n(network):
            for n in ActiveAction:
                if not ExistSynapse(ActivePerception[0],n):
                    Synapse(ActivePerception[0],n)
                    #print('Pleasure query: '+str(Pleasure_n.query_n(network))+'\n')
                    print(show(ActivePerception[0].name)+' to '+show(n.name)+
                          ' Pleasure: '+show(Pleasure_n.query_last_n(network)))
        elif Pain_n.query_last_n(network):
            for n in ActiveAction:
                if not ExistSynapse(ActivePerception[0],n,1):
                    Synapse(ActivePerception[0],n,1)
                    print(show(ActivePerception[0].name)+' inhibit '+show(n.name)+
                          ' Pain: '+show(Pain_n.query_last_n(network)))
    else:
        if Pleasure_n.query_last_n(network):
            shared_connections = SharedConnections(ActivePerception)
            if not shared_connections:
                perc_neuron = network.add_neuron(gensym('Perception'),den=len(ActivePerception))
                for n in range(len(ActivePerception)):
                    Synapse(ActivePerception[n],perc_neuron,n)
                    print(show(ActivePerception[n].name)+' to '+show(perc_neuron.name)+
                          ' Pleasure: '+show(Pleasure_n.query_last_n(network)))
            else:
                perc_neuron = shared_connections[0]
            for n in ActiveAction:
                    if not ExistSynapse(perc_neuron,n):
                        Synapse(perc_neuron,n)
                        print(show(perc_neuron.name)+' to '+show(n.name)+
                              ' Pleasure: '+show(Pleasure_n.query_last_n(network)))
        elif Pain_n.query_last_n(network):
            shared_connections = SharedConnections(ActivePerception)
            if not shared_connections:
                perc_neuron = network.add_neuron(gensym('Perception'),den=len(ActivePerception))
                for n in range(len(ActivePerception)):
                    Synapse(ActivePerception[n],perc_neuron,n)
                    print(show(ActivePerception[n].name)+' to '+show(perc_neuron.name)+
                          ' Pleasure: '+show(Pleasure_n.query_last_n(network)))
            else:
                perc_neuron = shared_connections[0]
            for n in ActiveAction:
                if not ExistSynapse(perc_neuron,n,1):
                    Synapse(perc_neuron,n,1)
                    print(show(perc_neuron.name)+' inhibit '+show(n.name)+
                              ' Pain: '+show(Pain_n.query_last_n(network)))
    if network.ntracing():
        network.ntrace()

def DoAction(x):
    if x.water<0.4:
        Drink_n.create_n(x.brain)
    elif x.food<0.4:
        Eat_n.create_n(x.brain)
    #elif x.water==1:
    #    Drink_gran.state.dendrites[1]=1#InhibitType_n(Drink_n).create_n(x.brain)
    #elif x.food==1:
    #    Eat_gran.state.dendrites[1]=1#InhibitType_n(Eat_n).create_n(x.brain)
    for n in [Green_gran,Blue_gran,Brown_gran]:
        n.mark_update()
    x.brain.run('f')
    #print('DoAction 1')
    #x.brain.display_history()
    LiveActions = [k for k in NeuralType2Action.keys() if k.query_last_n(x.brain)]
    if LiveActions == []:
        LiveActions = [random.choice([Eat_n,Drink_n,MoveLeft_n,MoveRight_n])]
    Act = random.choice(LiveActions)
    for a in [i for i in LiveActions if not i == Act]: InhibitType_n(a).create_n(x.brain)
    Act.create_n(x.brain)
    for n in [Green_gran,Blue_gran,Brown_gran]:
        n.mark_update()
    x.brain.run('f')
    #print('DoAction 2')
    #x.brain.display_history()
    if Act.query_last_n(x.brain):
        NeuralType2Action[Act](x)
        #Learn(x.brain)
    #print('Create '+show(Act))
    #x.brain.display_history()
    #for k in LiveActions:
    #    InhibitType_n(k).create_n(x.brain)
    #InhibitType_n(Pleasure_n).create_n(x.brain)
    #InhibitType_n(Pain_n).create_n(x.brain)
def DoEat(x):
    #Eat_n.create_n(x.brain)
    if Swamp.query(x.loc):
        x.food = x.food-0.1
        x.water = x.water-0.1
        InhibitType_n(Pleasure_n).create_n(x.brain)
        Pain_n.create_n(x.brain)
    elif Green.query(x.loc) and x.food<1:
        x.food = x.food+0.1
        InhibitType_n(Pain_n).create_n(x.brain)
        Pleasure_n.create_n(x.brain)
    else:
        #x.food = x.food-0.1
        #InhibitType_n(Pleasure_n).create_n(x.brain)
        #Pain_n.create_n(x.brain)
        InhibitType_n(Eat_n).create_n(x.brain)
        Act = random.choice([MoveLeft_n,MoveRight_n])
        Act.create_n(x.brain)
        NeuralType2Action[Act](x)
        #else:
            #InhibitType_n(Pleasure_n).create_n(x.brain)
            #Pain_n.create_n(x.brain)
            #Act = random.choice([Drink_n,MoveLeft_n,MoveRight_n])
            #Act.create_n(x.brain)
            #DoAction(x)
   #else:
        #Act = random.choice([MoveLeft_n,MoveRight_n])
        #Act.create_n(x.brain)
        #DoAction(x)
def DoDrink(x):
    #Drink_n.create_n(x.brain)
    if Swamp.query(x.loc):
        x.food = x.food-0.1
        x.water = x.water-0.1
        InhibitType_n(Pleasure_n).create_n(x.brain)
        Pain_n.create_n(x.brain)
    elif Blue.query(x.loc) and x.water<1:
        x.water = x.water+0.1
        InhibitType_n(Pain_n).create_n(x.brain)
        Pleasure_n.create_n(x.brain)
        #else:
        #    InhibitType_n(Pleasure_n).create_n(x.brain)
        #    Pain_n.create_n(x.brain)
        #    Act = random.choice([Eat_n,MoveLeft_n,MoveRight_n])
        #    Act.create_n(x.brain)
            #DoAction(x)
    else:
        #x.water = x.water-0.1
        #InhibitType_n(Pleasure_n).create_n(x.brain)
        #Pain_n.create_n(x.brain)
        InhibitType_n(Drink_n).create_n(x.brain)
        Act = random.choice([MoveLeft_n,MoveRight_n])
        Act.create_n(x.brain)
        NeuralType2Action[Act](x)
        #DoAction(x)

        
NeuralType2Action[Eat_n]=DoEat
NeuralType2Action[Drink_n]=DoDrink

In [15]:
def Cycle(x,learn,n):
    for n in range(n):
        print(n)
        if Alive(x):
            x.brain.nontrace()
            x.brain.ntrace()
            DoObserveEnv(x)
            x.brain.run('f')
            DoAction(x)
            learn(x.brain)
            x.brain.display_history()
            print('food: '+show(x.food))
            print('water: '+show(x.water))
            print('loc: '+show(x.loc)+'\n')
        else:
            print('Died')
            break


In [16]:
Cycle(maud,Learn2,50)

0
-----------  -  -  -  -
Green_n      1  0  1  1
Blue_n       0  0  0  0
Brown_n      0  0  0  0
Eat_n        0  0  0  0
Drink_n      0  0  0  0
MoveLeft_n   0  0  0  1
MoveRight_n  0  0  0  0
Pleasure_n   1  0  0  0
Pain_n       0  0  0  0
-----------  -  -  -  -
food: 0.88
water: 0.68
loc: loc4

1
-----------  -  -  -  -
Green_n      1  0  1  1
Blue_n       0  0  0  0
Brown_n      0  0  0  0
Eat_n        0  0  0  0
Drink_n      0  0  0  0
MoveLeft_n   1  0  0  1
MoveRight_n  0  0  0  0
Pleasure_n   0  0  0  0
Pain_n       0  0  0  0
-----------  -  -  -  -
food: 0.78
water: 0.5800000000000001
loc: loc3

2
-----------  -  -  -  -
Green_n      1  0  0  0
Blue_n       0  0  1  1
Brown_n      0  0  0  0
Eat_n        0  0  0  0
Drink_n      0  0  0  0
MoveLeft_n   1  0  0  1
MoveRight_n  0  0  0  0
Pleasure_n   0  0  0  0
Pain_n       0  0  0  0
-----------  -  -  -  -
food: 0.68
water: 0.4800000000000001
loc: loc2

3
Green_n to Perception0 Pleasure: False
Blue_n to Perception0 Pleasure: